In [1]:
# imports
import pandas as pd
import random
import time
from functions import vanilla_score
from functions import kippendorff_analysis
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Define a placeholder function for vanilla_score
def random_score(text):
    return random.randint(1, 5)  # Placeholder to return a random score

In [3]:
# read in text data --> text_data
text_data = pd.read_csv('Data/texts.csv').drop(columns='Q')
text_data['T'] = text_data['T'].str.replace('T', '').astype(int)
text_data

,T,Text,Source,Industry,Interest Category
0,1,Find Your Dream Sofa at Westwing: Transform Yo...,GenAI Tool,Consumer Goods and Retail,General Interest
1,2,Explore Sustainable and Stylish Backpacks at G...,GenAI Tool,Consumer Goods and Retail,General Interest
2,3,Designer clothes have always been one of the m...,Websearch,Consumer Goods and Retail,Professional Interest
3,4,Whether you're a seasoned gardener or just sta...,Websearch,Consumer Goods and Retail,General Interest
4,5,Driving the energy transition: Solutions for i...,Websearch,Energy and Environment,General Interest
5,6,"Chocolate, Sugar Snap and Orange Dessert Recip...",Websearch,Food and Beverage,Professional Interest
6,7,Large Format Cinematic Digital Film Sensor!\nB...,Websearch,Entertainment / Media and Education,Special Interest
7,8,N26 Smart—save and spend with confidence\nDisc...,Websearch,Financial Services and FinTech,Special Interest
8,9,CHANGE THE WAY YOU MONEY\nFor those who want m...,Websearch,Financial Services and FinTech,General Interest
9,10,ADUHELM is indicated for the treatment of Alzh...,Websearch,Healthcare and Pharmaceuticals,Professional Interest


In [7]:
# Data Generation - OpenAI Models 
import time

# Remove # for models to use
gpt_dict = {
    # "gpt35t": "gpt-3.5-turbo-0125",
    # "gpt4": "gpt-4-0613",
    # "gpt4t": "gpt-4-turbo-2024-04-09",
    # "gpt4o": "gpt-4o-2024-05-13",
    # "nemotron4340b":"nvidia/Nemotron-4-340B-Instruct"
}

duration = [['Model', 0, 1]]

for name in gpt_dict:
    times = [name]
    for temp in [0, 0.2, 0.4, 0.6, 0.8, 1]:
        model = gpt_dict.get(name)
        temperature = temp
        evaluations = 20
        texts = [1, 4, 6, 9, 10, 11, 14, 16, 19, 20]

        # create variables
        evaluator = 'vanilla_' + name + f"_{int(temp * 10):02}" # will contain score counts and descriptive statistics
        scores = evaluator + '_scores' # contains raw scores in wide format
        direction = 'Data_LLM/' + scores + '.csv'

        globals()[scores] = text_data[['T', 'Text']]
        globals()[scores] = globals()[scores].loc[globals()[scores]['T'].isin(texts)] # subset to only run for texts

        # collect and transform scores --> vanilla_GPT35_0_scores
        start_time = time.process_time()
        for i in range(1, evaluations+1): # create {evaluations} columns with scores
            # globals()[scores]['score ' + str(i)] = globals()[scores]['Text'].apply(lambda x: pd.Series(random_score(x)))
            globals()[scores]['score ' + str(i)] = globals()[scores]['Text'].apply(lambda x: pd.Series(vanilla_score(x, model=model, temperature=temperature, reason=False)))
        end_time = time.process_time()
        eval_time = end_time - start_time
        times.append(eval_time)

        globals()[scores] = globals()[scores].drop(columns=['Text'])
        globals()[scores] = globals()[scores].set_index('T').rename_axis(scores, axis='columns')
        globals()[scores]
        globals()[scores].to_csv(direction)

        
    duration.append(times)
    print(times)

duration = pd.DataFrame(duration[1:], columns=duration[0])
duration

['gpt4', 7.288472999999996, 7.190812000000001]


,Model,0,1
0,gpt4,7.288473,7.190812


In [6]:
# Data Generation - non-OpenAI Models
# Remove # for models to use
MODEL_MAPPING = {
    # "mistral7b": {"model": "mistralai/Mistral-7B-Instruct-v0.3", "provider": "deepinfra"},
    # "mixtral8x22b": {"model": "mistralai/Mixtral-8x22B-Instruct-v0.1", "provider": "deepinfra"},
    # "llama370b": {"model": "meta-llama/Meta-Llama-3-70B-Instruct", "provider": "deepinfra"},
    # "llama38b": {"model": "meta-llama/Meta-Llama-3-8B-Instruct", "provider": "deepinfra"},
    # "llama38b_private": {"model": "gpt-3.5-turbo", "provider": "ommax_vllm"}, # model name is faked for openai compatibility
    # "claude3opus": {"model": "claude-3-opus-20240229", "provider": "anthropic"},
    # "claude3sonnet": {"model": "claude-3-sonnet-20240229", "provider": "anthropic"},
    # "claude3haiku": {"model": "claude-3-haiku-20240307", "provider": "anthropic"},
    # "qwen2": {"model": "Qwen/Qwen2-72B-Instruct", "provider": "deepinfra"},
    # "phi3": {"model": "microsoft/Phi-3-medium-4k-instruct", "provider": "deepinfra"},
    # "wizardLM28x22b": {"model": "microsoft/WizardLM-2-8x22B", "provider": "deepinfra"},
    # "wizardLM27b": {"model": "microsoft/WizardLM-2-7B", "provider": "deepinfra"},
    # "gemma": {"model": "google/gemma-1.1-7b-it", "provider": "deepinfra"},
    # "nemotron4340b": {"model": "nvidia/Nemotron-4-340B-Instruct", "provider": "nvidia"},
}

duration = [['Model', 1]]


for model in MODEL_MAPPING:
    times = [model]
    for temp in [0, 0.2, 0.4, 0.6, 0.8, 1]: #
        model = model
        temperature = temp
        evaluations = 20
        texts = [1, 4, 6, 9, 10, 11, 14, 16, 19, 20]

        # create variables
        evaluator = 'vanilla_' + model + f"_{int(temp * 10):02}" # will contain score counts and descriptive statistics
        scores = evaluator + '_scores' # contains raw scores in wide format
        direction = 'Data_LLM/' + scores + '.csv'

        globals()[scores] = text_data[['T', 'Text']]
        globals()[scores] = globals()[scores].loc[globals()[scores]['T'].isin(texts)] # subset to only run for texts

        # collect and transform scores --> vanilla_GPT35_0_scores
        start_time = time.process_time()
        for i in range(1, evaluations+1): # create {evaluations} columns with scores
            # globals()[scores]['score ' + str(i)] = globals()[scores]['Text'].apply(lambda x: pd.Series(random_score(x)))
            globals()[scores]['score ' + str(i)] = globals()[scores]['Text'].apply(lambda x: pd.Series(vanilla_score(x, model=model, temperature=temperature, reason=False)))
        end_time = time.process_time()
        eval_time = end_time - start_time
        times.append(eval_time)

        globals()[scores] = globals()[scores].drop(columns=['Text'])
        globals()[scores] = globals()[scores].set_index('T').rename_axis(scores, axis='columns')
        globals()[scores]
        globals()[scores].to_csv(direction)

        
    duration.append(times)
    print(times)

duration = pd.DataFrame(duration[1:], columns=duration[0])
duration


['mistral7b', 4.474238999999997]


,Model,1
0,mistral7b,4.474239
